# Import libraries

In [1]:
import pandas as pd
import json
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk

# Import collocations dictionary

In [2]:
dictionary_path = './static/'
# Read collacations dictionary as list of dicts is a json file
with open(dictionary_path + 'diccionario_collocation.json', 'r') as f:
    collocations = json.load(f)
# Cast to dataframe and get only the columns
columns = ['COLLOCATION', 'COLLOCATION_VISUAL', 'RAIZ_COLLOCATION']
df_collocations = pd.DataFrame(collocations)[columns]
df_collocations.head()

,COLLOCATION,COLLOCATION_VISUAL,RAIZ_COLLOCATION
0,MOSTRAR INICIATIVA,MOSTRAR INICIATIVA,MOSTR INICI
1,TRABAJAR SOLISTAS,TRABAJAR CON SOLISTAS,TRABAJ SOLIST
2,FABRICAR INGREDIENTES,FABRICAR INGREDIENTES,FABRIC INGREDIENT
3,TECNICAS ASERRADO,TECNICAS DE ASERRADO,TECNIC ASERR
4,VULCANIZADO FRIO,VULCANIZADO EN FRIO,VULCANIZ FRI


# Legacy function find collocations

In [3]:
stopwords = stopwords.words('spanish')

def f_vector_collocation_json(texto):
    # importamos las librerías necesarias

    # Obtenemos las stopwords del idioma del módulo nltk
    

    # adverbios
    adverbios = ['ahora', 'antes', 'despues', 'tarde', 'luego', 'ayer', 'temprano', 'ya',
                 'todavia', 'anteayer', 'aun', 'pronto', 'hoy', 'aqui', 'ahi', 'alli',
                 'cerca', 'lejos', 'fuera', 'dentro', 'alrededor', 'aparte', 'encima',
                 'debajo', 'delante', 'detras', 'asi', 'bien', 'mal', 'despacio',
                 'deprisa', 'como', 'mucho', 'poco', 'muy', 'casi', 'todo', 'nada', 'algo',
                 'medio', 'demasiado', 'bastante', 'mas', 'menos', 'ademas', 'incluso',
                 'tambien', 'si', 'asimismo', 'no', 'tampoco', 'jamas', 'nunca', 'acaso',
                 'quiza', 'quizas', 'tal', 'vez', 'mejor']
    stopwords.extend(adverbios)

    # obtengo en un dataframe el diccionario de collocation
    with open(dictionary_path + 'diccionario_collocation.json',
              encoding="latin-1") as f:
        aux_collocations = f.read()
    diccionario_collocations = json.loads(aux_collocations)

    # obtengo en un dataframe el diccionario de equivalencias de collocation
    with open(dictionary_path + 'diccionario_equivalencias_collocation.json',
              encoding="latin-1") as f:
        aux_equiv = f.read()
    dic_equiv_collocations = json.loads(aux_equiv)

    vector_collocation = list()
    collocations_dic = list()
    collocations_equiv = list()

    for i in range(len(diccionario_collocations)):
        collocations_dic.append(diccionario_collocations[i]['RAIZ_COLLOCATION'])

    for i in range(len(dic_equiv_collocations)):
        collocations_equiv.append(dic_equiv_collocations[i]['RAIZ_COLLOCATION'])

    stemmer = SnowballStemmer("spanish")

    lista_palabras = str(texto).lower()
    palabras = lista_palabras.split()
    filtered_words = [word for word in palabras if word not in stopwords]

    table = {33: 32, 35: 32, 36: 32, 37: 32, 94: 32, 38: 32, 42: 32, 40: 32, 41: 32, 91: 32, 93: 32,
             123: 32, 125: 32, 58: 32, 59: 32, 44: 32, 47: 32, 60: 32, 62: 32, 92: 32, 124: 32, 96: 32,
             126: 32, 45: 32, 34: 32, 39: 32, 61: 32, 95: 32, 43: 32}

    palabras_encontradas = []

    for j in range(len(filtered_words)):
        sin_simbolo = filtered_words[j].replace("¡", "")
        sin_simbolo = sin_simbolo.replace("»", "")
        sin_simbolo = sin_simbolo.replace("«", "")
        sin_simbolo = sin_simbolo.replace("¿", "")
        sin_simbolo = sin_simbolo.replace("°", "")
        sin_simbolo = sin_simbolo.replace("º", "")
        sin_simbolo = sin_simbolo.replace("ª", "")
        sin_simbolo = sin_simbolo.replace("_x000d_", "")
        sin_simbolo = sin_simbolo.replace("@", "o")
        sin_simbolo = sin_simbolo.replace("á", "a")
        sin_simbolo = sin_simbolo.replace("à", "a")
        sin_simbolo = sin_simbolo.replace("À", "a")
        sin_simbolo = sin_simbolo.replace("Á", "a")
        sin_simbolo = sin_simbolo.replace("é", "e")
        sin_simbolo = sin_simbolo.replace("è", "e")
        sin_simbolo = sin_simbolo.replace("È", "e")
        sin_simbolo = sin_simbolo.replace("É", "e")
        sin_simbolo = sin_simbolo.replace("í", "i")
        sin_simbolo = sin_simbolo.replace("ì", "i")
        sin_simbolo = sin_simbolo.replace("Ì", "i")
        sin_simbolo = sin_simbolo.replace("Í", "i")
        sin_simbolo = sin_simbolo.replace("ó", "o")
        sin_simbolo = sin_simbolo.replace("ò", "o")
        sin_simbolo = sin_simbolo.replace("Ò", "o")
        sin_simbolo = sin_simbolo.replace("Ó", "o")
        sin_simbolo = sin_simbolo.replace("ú", "u")
        sin_simbolo = sin_simbolo.replace("ù", "u")
        sin_simbolo = sin_simbolo.replace("Ù", "u")
        sin_simbolo = sin_simbolo.replace("Ú", "u")
        sin_simbolo = sin_simbolo.replace("ñ", "ñ")
        sin_simbolo = sin_simbolo.replace("?", "")
        tratar = sin_simbolo.strip(r"!#$%^&*()[]{};:,./<>?\|`~-'=_·")
        tratar2 = tratar.translate(table)
        palabras = tratar2.split()
        tokens = []
        for palabra in palabras:
            if palabra not in stopwords:
                tokens.append(palabra.upper())
        for w in tokens:
            es_numero = w.isdigit()
            if len(w) > 2 and not es_numero:
                palabras_encontradas.append(w)
    collocations = []
    for n in [2, 3]:
        collocations.extend(nltk.ngrams(palabras_encontradas, n))

    for coll in collocations:
        if len(coll) == 2:
            coll_final = coll[0] + " " + coll[1]
        if len(coll) == 3:
            coll_final = coll[0] + " " + coll[1] + " " + coll[2]
        coll_sep = coll_final.split(sep=' ')
        coll_def = ""
        for k in range(len(coll_sep)):
            coll_def = coll_def + stemmer.stem(coll_sep[k]).upper() + " "
        coll_def = coll_def[:-1]
        if coll_def in collocations_dic and coll_final not in vector_collocation:
            vector_collocation.append(coll_final)
        elif coll_def in collocations_equiv and coll_final not in vector_collocation:
            vector_collocation.append(coll_final)

    value = {
        "vector_collocation": vector_collocation
    }

    return json.dumps(value)

In [4]:
texto = '''
Oferta de empleo de ingeniero de software con experiencia en desarrollo de aplicaciones web y móviles. Voz sobre protocolo de internet Necesaria experiencia en Python, Java y C++.
'''
lista_palabras = f_vector_collocation_json(texto)
print(lista_palabras)

{"vector_collocation": ["OFERTA EMPLEO"]}


# Clean text

In [5]:
import re
from unidecode import unidecode

expections = ["@", r"\."]

def clean_text(text):
    # ! EXCEPCIONES MANUALES!!!
    # Cast exceptions to "a"
    text = re.sub(r'@', 'a', text)
    # Cast from "." to ""
    text = re.sub(r'\.', '', text)
    
    # Trim the text
    text = text.strip()
    # Remove accents using unidecode, excluding 'ñ' and 'Ñ'
    text = ''.join(char if char in ('ñ', 'Ñ') else unidecode(char) for char in text)
    # Delete non-alphanumeric characters
    text = re.sub(r'[^\w\s]', ' ', text)
    # Delete additional spaces with regex
    text = re.sub(r'\s+', ' ', text).upper()
    return text
texto = '''
Oferta de empleo de ingeniero de software con experiencia en desarrollo de aplicaciones web y móviles. Voz sobre protocolo de internet Necesaria experiencia en Python, Java y C++.
'''
texto = clean_text(texto)
# Split the text by " "
list_words = texto.split(" ")
# Delete empty strings
list_words = list(filter(None, list_words))
# Change all stopwords to uppercase
stopwords = [word.upper() for word in stopwords]
# Delete stopwords
list_words = [word for word in list_words if word not in stopwords]
print(list_words)

['OFERTA', 'EMPLEO', 'INGENIERO', 'SOFTWARE', 'EXPERIENCIA', 'DESARROLLO', 'APLICACIONES', 'WEB', 'MOVILES', 'VOZ', 'PROTOCOLO', 'INTERNET', 'NECESARIA', 'EXPERIENCIA', 'PYTHON', 'JAVA', 'C']


# Get list of stems for text

In [6]:
def filter_words(text):
    # Split the text by " "
    list_words = text.split(" ")
    # Delete empty strings
    list_words = list(filter(None, list_words))
    # Filter all stopwords
    return [word for word in list_words if word not in stopwords]

stemmer = SnowballStemmer("spanish")

def get_list_stems(list_words):
    # Get stems
    return [stemmer.stem(word).upper() for word in list_words]

def get_n_gramas(list_stems, min_n_gramas, max_n_gramas):
    list_n_gramas = [" ".join(n_grama) for n in range(min_n_gramas, max_n_gramas + 1)
                     for n_grama in nltk.ngrams(list_stems, n)]
    return list_n_gramas

def calculate_forms(text, min_n_gramas=2, max_n_gramas=4):
    # First clean text
    text = clean_text(text)
    # Filter words
    list_words = filter_words(text)
    # Get list of stems
    list_stems = get_list_stems(list_words)
    # Get the n gramas
    list_n_gramas = get_n_gramas(list_stems, min_n_gramas, max_n_gramas)
    if list_n_gramas != []:
        return list_n_gramas
    else:
        return None
texto = '''
SENTIDO DE RESPONSABILIDAD
'''
list_n_gramas = calculate_forms(texto)
print(list_n_gramas)

None


# Calculo de FORMAS a partir de COLLOCATION

In [7]:
# Calculate all n_gramas for the column COLLOCATION_VISUAL in a new column
df_collocations['N_GRAMAS'] = df_collocations['COLLOCATION'].apply(calculate_forms)
# Compare the column N_GRAMAS with the column RAIZ_COLLOCATION
def compare_n_gramas(n_gramas, raiz_collocation):
    if n_gramas is not None and raiz_collocation in n_gramas:
        return True
    else:
        return False
# Create a new column with the result of the comparison
df_collocations['RESULT'] = df_collocations.apply(lambda row: compare_n_gramas(row['N_GRAMAS'], row['RAIZ_COLLOCATION']), axis=1)
# Show all non True values in the column RESULT
df_differences = df_collocations[df_collocations['RESULT'] != True][['N_GRAMAS', 'RAIZ_COLLOCATION']]
df_differences.head()

,N_GRAMAS,RAIZ_COLLOCATION
39,[CLASIF PESC],CLASIFICACION PESC
46,[TRASTORN ALIMENT],TRASTORN ALIMENTACION
459,[TECNIC COMPACT],TECNIC COMPACTACION
469,[CLASIF DEUD],CLASIFICACION DEUD
483,[MODELIZ RIESG],MODELIZACION RIESG


# Calculate the max total forms

In [8]:
def get_clean_text(text):
    # Clean text
    text = clean_text(text)
    # Filter words
    list_words = filter_words(text)
    return list_words

def get_all_forms(text):
    list_words = get_clean_text(text)
    return len(list_words)


texto = '''
Oferta de empleo de ingeniero de software con experiencia en desarrollo de aplicaciones web y móviles. Voz sobre protocolo de internet Necesaria experiencia en Python, Java y C++.
'''
df_collocations['N_FORMAS'] = df_collocations['COLLOCATION_VISUAL'].apply(get_all_forms)
# Order by N_FORMAS
df_collocations.sort_values(by=['N_FORMAS'], ascending=False, inplace=True)
df_collocations.head()

,COLLOCATION,COLLOCATION_VISUAL,RAIZ_COLLOCATION,N_GRAMAS,RESULT,N_FORMAS
8576,SABER MANEJAR COSTES EXPLOTACION,SABER MANEJAR COSTES DE EXPLOTACIÓN,SAB MANEJ COST EXPLOTACION,"[SAB MANEJ, MANEJ COST, COST EXPLOT, SAB MANEJ...",False,4
8546,LONG SHORT TERM MEMORY,LONG-SHORT TERM MEMORY,LONG SHORT TERM MEMORY,"[LONG SHORT, SHORT TERM, TERM MEMORY, LONG SHO...",True,4
8584,INTERES DESARROLLAR CARRERA PROFESIONAL,INTERÉS POR DESARROLLAR UNA CARRERA PROFESIONAL,INTER DESARROLL CARRER PROFESIONAL,"[INTER DESARROLL, DESARROLL CARRER, CARRER PRO...",True,4
8583,GESTION GRANDES GRUPOS EQUIPOS,GESTIÓN DE GRANDES GRUPOS O EQUIPOS,GESTION GRAND GRUP EQUIP,"[GESTION GRAND, GRAND GRUP, GRUP EQUIP, GESTIO...",True,4
8581,CREADOR BUEN AMBIENTE TRABAJO,CREADOR DE BUEN AMBIENTE DE TRABAJO,CREADOR BUEN AMBIENT TRABAJ,"[CREADOR BUEN, BUEN AMBIENT, AMBIENT TRABAJ, C...",True,4


# Export new COLLOCATIONS

In [10]:
print(df_collocations.columns)

Index(['COLLOCATION', 'COLLOCATION_VISUAL', 'RAIZ_COLLOCATION', 'N_GRAMAS',
       'RESULT', 'N_FORMAS'],
      dtype='object')


In [13]:
# Get only the columns COLLOCATION, COLLOCATION_VISUAL and N_GRAMAS
df_new_collocations = df_collocations[['COLLOCATION', 'COLLOCATION_VISUAL', 'N_GRAMAS']]
# Rename columns as LEMA, COLLOCATION and FORMAS
df_new_collocations.columns = ['LEMA', 'COLLOCATION', 'FORMAS']
# # Explode the dataframe by the column FORMAS
df_new_collocations = df_new_collocations.explode('FORMAS')
# # Drop duplicates
df_new_collocations.drop_duplicates(inplace=True)
# Drop empty strings
df_new_collocations = df_new_collocations[(df_new_collocations['FORMAS'] != '') & (~df_new_collocations['FORMAS'].isnull())]
print(df_new_collocations.shape)
# Get the number of words for each FORMAS
df_new_collocations['N_WORDS_FORMAS'] = df_new_collocations['FORMAS'].apply(lambda x: len(x.split(" ")))
# Get the number of words for each LEMA
df_new_collocations['N_WORDS_LEMA'] = df_new_collocations['LEMA'].apply(lambda x: len(x.split(" ")))
# Get all the registers with same number of N_WORDS_LEMA and N_WORDS_FORMAS
# df_new_collocations = df_new_collocations[df_new_collocations['N_WORDS_LEMA'] == df_new_collocations['N_WORDS_FORMAS']]
# Order alphabetically by FORMAS
df_new_collocations.sort_values(by=['FORMAS', 'N_WORDS_FORMAS'], ascending=True, inplace=True)
# Save the dataframe as csv
df_new_collocations.to_csv(dictionary_path + 'diccionario_collocation.csv', index=False)
# Show the dataframe
df_new_collocations.head(10)

(19278, 3)


,LEMA,COLLOCATION,FORMAS,N_WORDS_FORMAS,N_WORDS_LEMA
7385,ABBYY FINEREADER,ABBYY FINEREADER,ABBYY FINEREAD,2,2
2298,ABOGAR PERSONAS,ABOGAR POR OTRAS PERSONAS,ABOG PERSON,2,2
3680,ASESORAR ABONOS PLANTAS,ASESORAR SOBRE ABONOS PARA PLANTAS,ABON PLANT,2,3
2047,ABRASIVOS CHORRO,ABRASIVOS DE CHORRO,ABRAS CHORR,2,2
4050,ABRIR CUENTAS BANCO,ABRIR CUENTAS DE BANCO,ABRIR CUENT,2,3
4050,ABRIR CUENTAS BANCO,ABRIR CUENTAS DE BANCO,ABRIR CUENT BANC,3,3
8207,ABRIR GALERIAS CONDUCCIONES,ABRIR GALERIAS DE CONDUCCIONES,ABRIR GALERI,2,3
8207,ABRIR GALERIAS CONDUCCIONES,ABRIR GALERIAS DE CONDUCCIONES,ABRIR GALERI CONDUCCION,3,3
4594,ABROCHAR DISPOSITIVOS SEGURIDAD,ABROCHAR LOS DISPOSITIVOS DE SEGURIDAD,ABROCH DISPOSIT,2,3
4594,ABROCHAR DISPOSITIVOS SEGURIDAD,ABROCHAR LOS DISPOSITIVOS DE SEGURIDAD,ABROCH DISPOSIT SEGUR,3,3


# Find by forma

In [ ]:
forma = "GESTION DOCUMENT"
# Find the df_new_collocations with the FORMAS == forma
print(df_new_collocations[df_new_collocations['FORMAS'] == forma]["LEMA"])

5687         GESTIONAR DOCUMENTOS DIGITALES
6062    GESTIONAR DOCUMENTACION FABRICACION
2372                     GESTION DOCUMENTOS
Name: LEMA, dtype: object
